In [64]:
from bs4 import BeautifulSoup
import requests
import os
from pathlib import Path
import pandas as pd

path_prefix = "dump/"
base_url = "http://www.streamlinerschedules.com/"

In [12]:
def download_track_pages():
    track_pages = [f"concourse/track{i}" for i in range(1, 13)]
    for page in track_pages:
        Path(path_prefix + page).mkdir(exist_ok=True, parents=True)
        with open(f"{path_prefix}{page}/index.html", "wb+") as f:
            f.write(requests.get(f"{base_url}{page}/index.html").content)

check_file = Path(f"{path_prefix}/concourse/track1/index.html")

if check_file.exists() and check_file.stat().st_size > 0:
    print("Already downloaded")
else:
    # download_track_pages()
    pass

Already downloaded


In [27]:
track_pages = [f"concourse/track{i}/index.html" for i in range(1, 13)]

In [28]:
with open(path_prefix + track_pages[0], "rb") as f:
    soup = BeautifulSoup(f, "html.parser")


In [42]:
# Returns a list like "birmspecial194112.html", "carolinaspecial196410.html", etc
def get_train_links(soup):
    relative_links = []
    for a in soup.find_all("a"):
        href = a.get("href")
        if href is not None and ".." not in href and "http://" not in href:
            relative_links.append(href)
    return relative_links

def get_link(relative_link, page):
    page = page.rstrip("index.html")
    return f"{base_url}{page}{relative_link}"


In [46]:
def read_child_pages(page):
    with open(path_prefix + page, "rb") as f:
        soup = BeautifulSoup(f, "html.parser")
        links = get_train_links(soup)
        for link in links:
            with open(f"{path_prefix}{page.rstrip('index.html')}/{link}", "wb+") as f:
                f.write(requests.get(get_link(link, page)).content)

In [63]:
def parse_schedule(schedule_filename):
    with open(schedule_filename, "rb") as f:
        soup = BeautifulSoup(f, "html.parser")
    
    train_name = soup.find("h1", id="trainname").text
    print(train_name)

    # TODO: Date
    description_element = soup.find('h3', id='rrdate')

    table = soup.find('table')
    train_numbers = table.find_all("td", class_="trainnum")
    for train_number in train_numbers:
        print(train_number.text)
    rows = table.find_all("tr")





parse_schedule("dump/concourse/track1/birmspecial194112.html")

The Birmingham Special
 125
 176(3rd)
 17
 18
 6
 5
 35
 36
 47
 48


In [92]:
with open("dump/concourse/track1/birmspecial194112.html", "rb") as f:
    soup = BeautifulSoup(f, "html.parser")

train_name = soup.find("h1", id="trainname").text
print(train_name)

description_element = soup.find('h3', id='rrdate')
table = soup.find('table')

train_numbers = table.find_all("td", class_="trainnum")
for train_number in train_numbers:
    print(train_number.text)

The Birmingham Special
 125
 176(3rd)
 17
 18
 6
 5
 35
 36
 47
 48


In [101]:
stops = pd.DataFrame(columns=["train_number", "station", "time", "miles"])

rows = table.find_all("tr")
for i, header in enumerate(rows[1].find_all("td")):
    if header.text == "Miles":
        miles_index = i
        station_index = i + 1

for row in rows:
    if row.find("td", class_="trainnum") is not None:
        train_nums = [t.text.strip() for t in row.find_all("td", class_="trainnum")]
        continue
    if row.find("td", class_="times") is None:
        continue
    times = row.find_all("td", class_="times")

    miles = row.find_all("td")[miles_index].text
    station = row.find_all("td")[station_index].text

    for num, time in zip(train_nums, times):
        stops = stops._append({"train_number": num, "station": station, "time": time.text, "miles": miles}, ignore_index=True)
    
stops

,train_number,station,time,miles
0,125,"New York, NY (Penna. Sta.) (ET)",\r\n12 30P,0.0
1,176(3rd),"New York, NY (Penna. Sta.) (ET)",\r\n4 45P,0.0
2,125,"Newark, NJ",\r\n12 45P,10.0
3,176(3rd),"Newark, NJ",\r\n4 29P,10.0
4,125,"Trenton, NJ",\r\n1 31P,58.1
...,...,...,...,...
113,48,"Laurel, MS",11 04A,209.1
114,47,"Hattiesburg, MS",\r\n5 43P,238.0
115,48,"Hattiesburg, MS",10 30A,238.0
116,47,"New Orleans, LA (Terminal\r\nSta.) (CT)",\r\n8 15P,354.6


In [104]:
stops.station

0               New York, NY (Penna. Sta.) (ET)
1               New York, NY (Penna. Sta.) (ET)
2                                    Newark, NJ
3                                    Newark, NJ
4                                   Trenton, NJ
                         ...                   
113                                  Laurel, MS
114                             Hattiesburg, MS
115                             Hattiesburg, MS
116     New Orleans, LA (Terminal\r\nSta.) (CT)
117     New Orleans, LA (Terminal\r\nSta.) (CT)
Name: station, Length: 118, dtype: object

In [91]:
rows[2].find_all("td", class_="times")

[<td class="ttbody times" style="font-weight: bold;">
 12 30P</td>,
 <td class="ttbody times" style="font-weight: bold;">
 4 45P</td>]

In [103]:
for page in track_pages:
    with open(path_prefix + page, "rb") as f:
        soup = BeautifulSoup(f, "html.parser")
        links = get_train_links(soup)
        for link in links:
            file_location = f"{path_prefix}{page.rstrip('index.html')}{link}"
            soup = BeautifulSoup(open(file_location, "rb"), "html.parser")
            train_name = soup.find("h1", id="trainname").text
            print(train_name)

The Birmingham Special
The Carolina Special
The City of Memphis
The City of New Orleans


AttributeError: 'NoneType' object has no attribute 'text'